In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import permutations
from pathlib import Path

from dolfin import XDMFFile
from dolfin import plot, lhs, rhs
from dolfin import TestFunction, TrialFunctions
from dolfin import RectangleMesh, Point, FiniteElement, MixedElement, FunctionSpace, Constant, Function, Measure
from dolfin import TestFunctions, TrialFunctions, TrialFunction
from dolfin import inner, grad, dx, ds, assemble
from dolfin import solve
from dolfin import FunctionSpace
from dolfin import split


from pantarei.boundary import DirichletBoundary, process_dirichlet#, process_boundary_forms, process_dirichlet
from pantarei.domain import Domain
from pantarei.io import TimeSeriesStorage

from multirat.base.meshprocessing import Domain
from multirat.base.boundary import RobinBoundary, process_boundary_forms
from multirat.computers import BaseComputer
from multirat.parameters import get_base_parameters, compute_parameters, make_dimless, PARAMETER_UNITS
from multirat.parameters import distribute_subset_parameters, print_quantities, get_interface_parameter

In [3]:
def create_mesh(n, x0=-1., y0=-1., x1=1., y1=1.):
    return Domain(
        mesh=RectangleMesh(Point(x0, y0), Point(x1, y1), n, n),
        subdomains=None,
        boundaries=None,
    )


def hydraulic_conductivity(params):
    k = params["permeability"]
    mu = params["viscosity"]
    return {key: k[key] / mu[key] for key in k}


def solve_pressure(domain, V, compartments, boundaries, params):
    p = TrialFunctions(V)
    q = TestFunction(V)
    K = {key: Constant(val) for key, val in hydraulic_conductivity(params).items()}
    T = get_interface_parameter(params["convective_fluid_transfer"], compartments, True)
    T = {key: Constant(val) for key, val in T.items()}

    imap = {label: idx for idx, label in enumerate(compartments)}
    F = 0.0
    for j in compartments:
        F += (K[j] * inner(grad(p[imap[j]]), grad(q[imap[j]]))
            - sum([ T[(i, j)]*(p[imap[i]] - p[imap[j]])*q[imap[j]] for i in compartments if i != j])
        ) * dx
    bcs = []
    for i in compartments:
        bcs.extend(process_dirichlet(domain, V.sub(imap[i]), boundaries[i]))
        F += process_boundary_forms(p[imap[i]], q[imap[i]], domain, boundaries[i])
    A = assemble(lhs(F))
    b = assemble(rhs(F))

    P = Function(V, name="pressure")
    for bc in bcs:
        bc.apply(A, b)
        solve(A, P.vector(), b)
        
    return P

In [14]:
compartments = ['ecs', 'pvs_arteries', 'pvs_veins']
base = distribute_subset_parameters(get_base_parameters())
params = make_dimless(compute_parameters(base), PARAMETER_UNITS)
print_quantities(params, 40)
get_interface_parameter(params["convective_fluid_transfer"], compartments, True)


permeability
  ecs                                    : 2.000e-11
  pvs_arteries                           : 1.000e-11
  pvs_capillaries                        : 3.519e-13
  pvs_veins                              : 6.515e-09
  arteries                               : 2.087e-12
  capillaries                            : 1.739e-12
  veins                                  : 1.044e-11
viscosity
  blood                                  : 2.670e-03
  csf                                    : 7.000e-04
  arteries                               : 2.670e-03
  capillaries                            : 2.670e-03
  veins                                  : 2.670e-03
  ecs                                    : 7.000e-04
  pvs_arteries                           : 7.000e-04
  pvs_capillaries                        : 7.000e-04
  pvs_veins                              : 7.000e-04
porosity
  ecs                                    : 1.400e-01
  arteries                               : 6.580e-03
  pvs_arteries

{('ecs', 'pvs_arteries'): 0.0007895385008901647,
 ('ecs', 'pvs_veins'): 0.0007031827273553029,
 ('pvs_arteries', 'ecs'): 0.0007895385008901647,
 ('pvs_arteries', 'pvs_veins'): 0.0,
 ('pvs_veins', 'ecs'): 0.0007031827273553029,
 ('pvs_veins', 'pvs_arteries'): 0.0}

In [15]:
base = distribute_subset_parameters(get_base_parameters())
domain = create_mesh(40)
compartments = ["pvs_arteries", "pvs_veins", "ecs"]

P1 = FiniteElement('CG', domain.mesh.ufl_cell(), 1)
el = MixedElement([P1]* len(compartments))
V = FunctionSpace(domain.mesh, el)
boundaries = {
    "pvs_arteries": [RobinBoundary(Constant(1e-1), Constant(1.0), "everywhere")],
    "pvs_veins": [DirichletBoundary(Constant(0.0), "everywhere")],
    "ecs": [DirichletBoundary(Constant(0.5), "everywhere")]
}
results_path = "../results/pressure"
params = make_dimless(compute_parameters(base), PARAMETER_UNITS)
p = solve_pressure(domain, V, compartments, boundaries, params)

storage = TimeSeriesStorage("w", "test/", mesh=domain.mesh, V=V)
storage.write(p, 0.0)
storage.close()

visual = TimeSeriesStorage("r", storage.filepath)
visual.to_xdmf(compartments)
visual.close()

In [16]:
from dolfin import Expression, Function, assign, project

In [17]:
params

{'permeability': {'ecs': 2e-11,
  'pvs_arteries': 1.000082101127542e-11,
  'pvs_capillaries': 3.518807392856166e-13,
  'pvs_veins': 6.5148205444879875e-09,
  'arteries': 2.0873485024539508e-12,
  'capillaries': 1.7394570853782927e-12,
  'veins': 1.0436742512269756e-11},
 'viscosity': {'blood': 0.00267,
  'csf': 0.0007,
  'arteries': 0.00267,
  'capillaries': 0.00267,
  'veins': 0.00267,
  'ecs': 0.0007,
  'pvs_arteries': 0.0007,
  'pvs_capillaries': 0.0007,
  'pvs_veins': 0.0007},
 'porosity': {'ecs': 0.14,
  'arteries': 0.00658,
  'pvs_arteries': 0.0006000000000000001,
  'capillaries': 0.00329,
  'pvs_capillaries': 0.00030000000000000003,
  'veins': 0.02303,
  'pvs_veins': 0.0021},
 'convective_fluid_transfer': {('ecs', 'pvs_arteries'): 0.0007895385008901647,
  ('pvs_arteries', 'arteries'): 9.951878777943514e-06,
  ('ecs', 'pvs_capillaries'): 3.5913889195386958e-06,
  ('pvs_capillaries', 'capillaries'): 3.3114590279571995e-05,
  ('ecs', 'pvs_veins'): 0.0007031827273553029,
  ('pvs_vei

In [45]:
def solve_solute(c0, p, dt, T, domain, V, compartments, boundaries, solute, params):
    c = TrialFunctions(V)
    w = TestFunction(V)
    phi = {key: Constant(val) for key, val in params["porosity"].items()}
    D = {key: Constant(val) for key, val in params[f"effective_diffusion"].items()}
    K = {key: Constant(val) for key, val in hydraulic_conductivity(params).items()}
    G = get_interface_parameter(params["convective_solute_transfer"], compartments, True)
    G = {key: Constant(val) for key, val in G.items()}
    L = get_interface_parameter(params["diffusive_solute_transfer"], compartments, True)
    L = {key: Constant(val) for key, val in L.items()}
    
    imap = {label: idx for idx, label in enumerate(compartments)}
    F = 0.0
    for j in compartments:
        idx_j = imap[j]
        F += (c[idx_j] - c0[idx_j]) * w[idx_j] * dx
        F += dt * (inner(D[j] * grad(c[idx_j]) + K[j]/phi[j] * c[idx_j] * grad(p[idx_j]), grad(w[idx_j]))) * dx
        sj = 0.0
        for i in compartments:
            if i == j:
                continue
            idx_i = imap[i]
            sj += (L[(i, j)] * (c[idx_i] - c[idx_j])
                   + G[(i, j)] * (p[idx_i] - p[idx_j]) * (0.5 * (c[idx_i] + c[idx_j]))
                ) / phi[j]
        F -= dt * sj * w[idx_j] * dx
        
    bcs = []
    for i in compartments:
        bcs.extend(process_dirichlet(domain, V.sub(imap[i]), boundaries[i]))
        F += process_boundary_forms(c[imap[i]], w[imap[i]], domain, boundaries[i])
    a = lhs(F)
    l = rhs(F)
    A = assemble(a)    
    C = Function(V, name="concentration")
    C0 = project(c0, V)
    storage = TimeSeriesStorage("w", "test/concentration/", mesh=domain.mesh, V=V)
    
    t = 0.0
    storage.write(C0, t)
    while t < T:
        print(f't = {t:.3f}', end="\r")
        b = assemble(l)
        for bc in bcs:
            bc.apply(A, b)
            solve(A, C.vector(), b)
        C0.assign(C)
        t += dt
        storage.write(C, t)
    storage.close()

    visual = TimeSeriesStorage("r", storage.filepath)
    visual.to_pvd(compartments)
    visual.close()        
    return C

In [46]:
from dolfin import Expression

In [50]:
N = 600
T = 4 * 3600
dt  = T / N

c_init = Expression("exp(-(pow(x[0] - c[0], 2) + pow(x[1]-c[1], 2)) / (length * length))",
                    length=Constant(0.2), c=Constant((0.5, 0.0)), degree=2)

c0 = Function(V)
assign(c0.sub(0), project(c_init, V.sub(0).collapse()))

In [51]:
solute_bcs = {
    "pvs_arteries": [DirichletBoundary(Constant(0.0), "everywhere")],
    "pvs_veins": [DirichletBoundary(Constant(0.0), "everywhere")],
    "ecs": [DirichletBoundary(Constant(0.0), "everywhere")]
}

In [52]:
solve_solute(c0, p, dt, T, domain, V, compartments, solute_bcs, 'inulin', params)

Calling FFC just-in-time (JIT) compiler, this may take some time.


Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 127), MixedElement(FiniteElement('Lagrange', triangle, 1), FiniteElement('Lagrange', triangle, 1), FiniteElement('Lagrange', triangle, 1))), 4394)

In [12]:
print_quantities(pressure_params, offset=37)

NameError: name 'pressure_params' is not defined

In [7]:
print_quantities(solute_params, offset=37)

NameError: name 'print_quantities' is not defined